In [1]:
from PIL import Image  as im
import io
import numpy as np
import gmpy2
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA1
from Crypto.Random import get_random_bytes
import requests as R
import time
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding

def getPhoto():
    i = 0
    data = b""
    while i<5:
        req = R.get('https://thispersondoesnotexist.com', timeout=60)
        if req.status_code == 200 and req.headers['Content-Type'][:5]== "image":
            data = req.content
            break
        time.sleep(5)
        i+=1
        print(data)
    return data

def TrandomSequence(n):
    img_bytes = getPhoto()
    img = im.open(io.BytesIO(img_bytes))


    out = img.convert('L')
    size = 364
    width, height = out.size
    left = (width - size) / 2
    top = (height - size) / 2
    right = (width + size) / 2
    bottom = (height + size) / 2

    out = out.crop((left, top, right, bottom))

    out = np.array(out)
    width, height = out.shape
    for i in range(8):
        for y in range(height-1):
            for x in range(width-1):
                out[y+1][x+1] = out[(x+2*y)%size][(x+y)%size]

    greyOut = np.asarray(out)

    for y in range(height):
            for x in range(width):
                if(out[x][y] <= 127):
                    out[x][y] = 0
                else:
                    out[x][y] = 1

    for x in np.arange(0,out.shape[0],4):
        for y in np.arange(0,out.shape[1],4):
            if x+3 < out.shape[0] and y+3 < out.shape[1]:
                sum = np.sum(out[x:x+4,y:y+4])
                if sum%2 == 0:
                    out[x:x+4,y:y+4] = 0
                elif sum%2 != 0:
                    out[x:x+4,y:y+4] = 1

    newOut = np.resize(out, (int(size/4),int(size/4)))

    tempy =0
    for x in np.arange(0,out.shape[1],4):
        tempx =0 
        for y in np.arange(0,out.shape[0],4):
            if x+3 < out.shape[0] and y+3 < out.shape[1]:
                sum = np.sum(out[x:x+4,y:y+4])
                if sum == 16:
                    newOut[tempx][tempy] = 1
                elif sum == 0:
                    newOut[tempx][tempy] = 0
            tempx+=1
        tempy+=1

    solution=[[] for i in range(width+height-1)]
    width, height = newOut.shape
    for i in range(width):
        for j in range(height):
            sum=i+j
            if(sum%2 ==0):
    
                #add at beginning
                solution[sum].insert(0,int(newOut[i][j]))
            else:
    
                #add at end of the list
                solution[sum].append(int(newOut[i][j]))

    solution_value = [j for i in solution for j in i]
    rand_bytes = bytearray()
    numeric_string = ""
    for i in solution_value:
        numeric_string += str(i)
        if(len(numeric_string) == 8):
            decimal_int = int(gmpy2.mpz(numeric_string, base=2))
            numeric_string = ""
            rand_bytes.append(decimal_int)
    print(1)
    
    return (rand_bytes)[:n]


#----------------------------------code here generates the pair of public and private keys, also saves them to file------------------------------------------------
# key_length = 1024

# key = RSA.generate(key_length, TrandomSequence)
# private_key = key.export_key()
# public_key = key.publickey().export_key()

# # Zapis kluczy do plików
# with open('private_key.pem', 'wb') as f:
#     f.write(private_key)

# with open('public_key.pem', 'wb') as f:
#     f.write(public_key)

#----------------------------------code here reads the keys from files, hash the message-> sign the mess using priv key next it hashes the same message,-----------------------------------------
#----------------------------------compares it with the decrypted signed mess using public key---------------------------------------------------------------------------------------------------
# Read private key from file
with open("private_key.pem", "rb") as private_key_file:
    private_key = serialization.load_pem_private_key(
        private_key_file.read(),
        password=None,
        backend=default_backend()
    )

# Message to sign
message = b"Hello, world!"

# Sign the message using the private key
signature = private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

# Save the signature to a file
with open("signature.bin", "wb") as signature_file:
    signature_file.write(signature)

# Read public key from file
with open("public_key.pem", "rb") as public_key_file:
    public_key = serialization.load_pem_public_key(
        public_key_file.read(),
        backend=default_backend()
    )

# Verify the signature using the public key
message = b"Hello, world!"
try:
    with open("signature.bin", "rb") as signature_file:
        signature = signature_file.read()
        public_key.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("messages are the same and the signature is Valid.")
except:
    print("messages are not the same or the signature is InValid.")


messages are the same and the signature is Valid.
